In [70]:
import pandas as pd
import os
from twarc import Twarc
import glob
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import itertools
import seaborn as sn
from textblob import TextBlob

# warning
import warnings
warnings.filterwarnings("ignore")

In [5]:
twitter_consumer_key = 'ZjhIda3R2URBVWLIa6y7AZNwZ'
twitter_consumer_secret = '9BRczF1KCEQlHvZWdlJPYFhhuEkRqNe5mKBdNgWvKQXdzOx00b'
twitter_access_token = '3243547069-9qDXRsMNaJSfCwYg2yIMRDyhSoqMXaM2Da6oxKK'
twitter_access_secret = 'Ota9ggjkJM1S14kPc8IPX3gRYqJGMmfqQ6zR3hoR1nPKX'

In [6]:
# Create the twarc object using the credentials above to hydrate tweets below
# t = Twarc(consumer_key=twitter_consumer_key, consumer_secret=twitter_consumer_secret, 
#           access_token=twitter_access_token, access_token_secret=twitter_access_secret)
# t

## Code for hydrating (no need to run)

In [7]:
# For each cleaned CSV of tweet ids, hydrate it & create a df. Then append it to a larger df containing all the
# tweets that have been hydrated so far.

# Running this cell takes close to 45 minutes, because there are 251,395 total tweets to hydrate, and
# the Twitter API hydrates at a rate of 360,000 tweets per hour.

# NOTE: This cell doesn't need to be run because the tweets have already been hydrated. This cell
# hydrates all the tweets into one massive df, and then after cleaning the df a bit,
# it is converted into a final CSV file.

# tweets_df = pd.DataFrame()
# path = "data/clean_data/*.csv"
# for clean_data_csv in glob.glob(path):
#     tweet_list = list(t.hydrate(open(clean_data_csv)))
#     tweet_df = pd.DataFrame(data=tweet_list)
#     tweets_df = tweets_df.append(tweet_df)

# Old - code used to create the slightly cleaned CSV

In [60]:
# Take columns that we think we will use
tw = tweets.loc[:, ['created_at', 'full_text', 'geo', 'truncated', 'display_text_range', 'user', 'id', 'entities', 'possibly_sensitive', 'lang']]
tw.head(1)

,created_at,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,lang
0,2020-03-19 19:53:17,"#statewaterheaters #getitin #corona #keepingpeoplesafe #plumber #pickeringtonohio new water heater swap out @ Pickerington, Ohio https://t.co/fMcPtH88m4","{'type': 'Point', 'coordinates': [39.8841, -82.759]}","{'id': '3995cc1483801d24', 'url': 'https://api.twitter.com/1.1/geo/id/3995cc1483801d24.json', 'place_type': 'city', 'name': 'Pickerington', 'full_name': 'Pickerington, OH', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-82.797752, 39.841431], [-82.684335, 39.841431], [-82.684335, 39.939034], [-82.797752, 39.939034]]]}, 'attributes': {}}",False,"[0, 152]","{'id': 169711005, 'id_str': '169711005', 'name': 'David A Hesson', 'screen_name': 'hessonplumbing', 'location': 'ohio', 'description': '', 'url': 'https://t.co/uks1jwtN2r', 'entities': {'url': {'urls': [{'url': 'https://t.co/uks1jwtN2r', 'expanded_url': 'http://hessonplumbing.com', 'display_url': 'hessonplumbing.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 18, 'friends_count': 19, 'listed_count': 0, 'created_at': 'Fri Jul 23 00:18:48 +0000 2010', 'favourites_count': 28, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 696, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': True, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1284683783002902529/yyxQR0Hr_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1284683783002902529/yyxQR0Hr_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/169711005/1542916153', 'profile_image_extensions_alt_text': None, 'profile_banner_extensions_alt_text': None, 'profile_link_color': '1B95E0', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': False, 'default_profile': False, 'default_profile_image': False, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}",1240728065983959040,"{'hashtags': [{'text': 'statewaterheaters', 'indices': [0, 18]}, {'text': 'getitin', 'indices': [19, 27]}, {'text': 'corona', 'indices': [28, 35]}, {'text': 'keepingpeoplesafe', 'indices': [36, 54]}, {'text': 'plumber', 'indices': [55, 63]}, {'text': 'pickeringtonohio', 'indices': [64, 81]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/fMcPtH88m4', 'expanded_url': 'https://www.instagram.com/p/B97XqQvJJJD/?igshid=h6pxa1d1cjr7', 'display_url': 'instagram.com/p/B97XqQvJJJD/…', 'indices': [129, 152]}]}",False,en


In [61]:
tw['truncated'] = tw.truncated.astype(bool)
tw['possibly_sensitive'] = tw.possibly_sensitive.astype(bool)

In [62]:
tw.isna().sum()

created_at              0
full_text               0
geo                   125
place                 615
truncated               0
display_text_range      4
user                    2
id                      0
entities                4
possibly_sensitive      0
lang                    4
dtype: int64

In [63]:
tw.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251395 entries, 0 to 251394
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   created_at          251395 non-null  object
 1   full_text           251395 non-null  object
 2   geo                 251270 non-null  object
 3   place               250780 non-null  object
 4   truncated           251395 non-null  bool  
 5   display_text_range  251391 non-null  object
 6   user                251393 non-null  object
 7   id                  251395 non-null  object
 8   entities            251391 non-null  object
 9   possibly_sensitive  251395 non-null  bool  
 10  lang                251391 non-null  object
dtypes: bool(2), object(9)
memory usage: 1.3 GB


In [64]:
# Export the df of tweets as a CSV file in the data directory
tw.to_csv(("../data/tweets_slightly_cleaned.csv"), index=False)

In [65]:
pd.read_csv('../data/tweets_slightly_cleaned.csv').head()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,created_at,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,lang
0,2020-03-19 19:53:17,"#statewaterheaters #getitin #corona #keepingpeoplesafe #plumber #pickeringtonohio new water heater swap out @ Pickerington, Ohio https://t.co/fMcPtH88m4","{'type': 'Point', 'coordinates': [39.8841, -82.759]}","{'id': '3995cc1483801d24', 'url': 'https://api.twitter.com/1.1/geo/id/3995cc1483801d24.json', 'place_type': 'city', 'name': 'Pickerington', 'full_name': 'Pickerington, OH', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-82.797752, 39.841431], [-82.684335, 39.841431], [-82.684335, 39.939034], [-82.797752, 39.939034]]]}, 'attributes': {}}",False,"[0, 152]","{'id': 169711005, 'id_str': '169711005', 'name': 'David A Hesson', 'screen_name': 'hessonplumbing', 'location': 'ohio', 'description': '', 'url': 'https://t.co/uks1jwtN2r', 'entities': {'url': {'urls': [{'url': 'https://t.co/uks1jwtN2r', 'expanded_url': 'http://hessonplumbing.com', 'display_url': 'hessonplumbing.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 18, 'friends_count': 19, 'listed_count': 0, 'created_at': 'Fri Jul 23 00:18:48 +0000 2010', 'favourites_count': 28, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 696, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': True, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1284683783002902529/yyxQR0Hr_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1284683783002902529/yyxQR0Hr_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/169711005/1542916153', 'profile_image_extensions_alt_text': None, 'profile_banner_extensions_alt_text': None, 'profile_link_color': '1B95E0', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': False, 'default_profile': False, 'default_profile_image': False, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}",1240728065983959040,"{'hashtags': [{'text': 'statewaterheaters', 'indices': [0, 18]}, {'text': 'getitin', 'indices': [19, 27]}, {'text': 'corona', 'indices': [28, 35]}, {'text': 'keepingpeoplesafe', 'indices': [36, 54]}, {'text': 'plumber', 'indices': [55, 63]}, {'text': 'pickeringtonohio', 'indices': [64, 81]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/fMcPtH88m4', 'expanded_url': 'https://www.instagram.com/p/B97XqQvJJJD/?igshid=h6pxa1d1cjr7', 'display_url': 'instagram.com/p/B97XqQvJJJD/…', 'indices': [129, 152]}]}",False,en
1,2020-03-19 19:53:46,"""ain't no humans outside! (corona!)"" 😂😂😂🤣 @ Cleveland, Ohio https://t.co/1i7cIsrITb","{'type': 'Point', 'coordinates': [41.4996, -81.6937]}","{'id': '0eb9676d24b211f1', 'url': 'https://api.twitter.com/1.1/geo/id/0eb9676d24b211f1.json', 'place_type': 'city', 'name': 'Cleveland', 'full_name': 'Cleveland, OH', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-81.877771, 41.392684], [-81.5331634, 41.392684], [-81.5331634, 41.599195], [-81.877771, 41.599195]]]}, 'attributes': {}}",False,"[0, 83]","{'id': 29464656, 'id_str': '29464656', 'name': '®oSs', 'screen_name': 'Mycox_Little', 'location': 'Simple Pleasures Lane', 'description': ""sarcastic as fuck dude I'm NOT lookin 4 nothin jus keepin up w/& meetin new folks welcome 2 MY 🌎"", 'url': 'https://t.co/zQjwMvzkyr', 'entities': {'url': {'urls': [{'url': 'https://t.co/zQjwMvzk

## Code when working with group Fri 12/4 on `geo` column

In [53]:
tweets = pd.read_csv('../../data/tweets_slightly_cleaned.csv')
pd.set_option('display.max_colwidth', None)
tweets.head(2)

,created_at,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,lang
0,2020-03-19 19:53:17,"#statewaterheaters #getitin #corona #keepingpeoplesafe #plumber #pickeringtonohio new water heater swap out @ Pickerington, Ohio https://t.co/fMcPtH88m4","{'type': 'Point', 'coordinates': [39.8841, -82.759]}","{'id': '3995cc1483801d24', 'url': 'https://api.twitter.com/1.1/geo/id/3995cc1483801d24.json', 'place_type': 'city', 'name': 'Pickerington', 'full_name': 'Pickerington, OH', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-82.797752, 39.841431], [-82.684335, 39.841431], [-82.684335, 39.939034], [-82.797752, 39.939034]]]}, 'attributes': {}}",False,"[0, 152]","{'id': 169711005, 'id_str': '169711005', 'name': 'David A Hesson', 'screen_name': 'hessonplumbing', 'location': 'ohio', 'description': '', 'url': 'https://t.co/uks1jwtN2r', 'entities': {'url': {'urls': [{'url': 'https://t.co/uks1jwtN2r', 'expanded_url': 'http://hessonplumbing.com', 'display_url': 'hessonplumbing.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 18, 'friends_count': 19, 'listed_count': 0, 'created_at': 'Fri Jul 23 00:18:48 +0000 2010', 'favourites_count': 28, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 696, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': True, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1284683783002902529/yyxQR0Hr_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1284683783002902529/yyxQR0Hr_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/169711005/1542916153', 'profile_image_extensions_alt_text': None, 'profile_banner_extensions_alt_text': None, 'profile_link_color': '1B95E0', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'has_extended_profile': False, 'default_profile': False, 'default_profile_image': False, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}",1240728065983959040,"{'hashtags': [{'text': 'statewaterheaters', 'indices': [0, 18]}, {'text': 'getitin', 'indices': [19, 27]}, {'text': 'corona', 'indices': [28, 35]}, {'text': 'keepingpeoplesafe', 'indices': [36, 54]}, {'text': 'plumber', 'indices': [55, 63]}, {'text': 'pickeringtonohio', 'indices': [64, 81]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/fMcPtH88m4', 'expanded_url': 'https://www.instagram.com/p/B97XqQvJJJD/?igshid=h6pxa1d1cjr7', 'display_url': 'instagram.com/p/B97XqQvJJJD/…', 'indices': [129, 152]}]}",False,en
1,2020-03-19 19:53:46,"""ain't no humans outside! (corona!)"" 😂😂😂🤣 @ Cleveland, Ohio https://t.co/1i7cIsrITb","{'type': 'Point', 'coordinates': [41.4996, -81.6937]}","{'id': '0eb9676d24b211f1', 'url': 'https://api.twitter.com/1.1/geo/id/0eb9676d24b211f1.json', 'place_type': 'city', 'name': 'Cleveland', 'full_name': 'Cleveland, OH', 'country_code': 'US', 'country': 'United States', 'contained_within': [], 'bounding_box': {'type': 'Polygon', 'coordinates': [[[-81.877771, 41.392684], [-81.5331634, 41.392684], [-81.5331634, 41.599195], [-81.877771, 41.599195]]]}, 'attributes': {}}",False,"[0, 83]","{'id': 29464656, 'id_str': '29464656', 'name': '®oSs', 'screen_name': 'Mycox_Little', 'location': 'Simple Pleasures Lane', 'description': ""sarcastic as fuck dude I'm NOT lookin 4 nothin jus keepin up w/& meetin new folks welcome 2 MY 🌎"", 'url': 'https://t.co/zQjwMvzkyr', 'entities': {'url': {'urls': [{'url': 'https://t.co/zQjwMvzk

In [54]:
len(tweets['geo'])

251395

In [55]:
any(tweets['geo'].isna())

True

In [60]:
def remove_na(df, colname):
    if any(df[colname].isna()):
        return df[df[colname].notna()]
    return df

In [58]:
tweets = remove_na(tweets, 'geo')

In [63]:
tweets = tweets.astype({'created_at': 'datetime64[ns]'})

In [64]:
# Get tfidf data
def get_tfidf_matrix(df, colname):
    text_list = df[colname]
    tf = TfidfVectorizer(analyzer='word', 
                     ngram_range=(1,3), 
                     min_df = 0, 
                     stop_words = 'english')
    
    tfidf_matrix = tf.fit_transform(text_list)
    return (tf, tfidf_matrix)

In [65]:
any(tweets.full_text.isna())

False

In [66]:
tweets = remove_na(tweets, 'full_text')

In [73]:
top_1k = tweets.loc[:1000, :]

In [75]:
tf, tfidf_matrix = get_tfidf_matrix(top_1k, 'full_text')

In [76]:
tf.get_feature_names()[:5]

['000', '000 rounds', '000 rounds think', '01pm', '01pm sunset']